In [31]:
import requests
import time
import json


API_KEY = ''

url = 'https://na1.api.riotgames.com/tft/league/v1/challenger?queue=RANKED_TFT&api_key=RGAPI-5d1cd49b-e13e-4a0f-b653-bc456de90fcd'

headers = {
    "X-Riot-Token": API_KEY
}

response = requests.get(url = url , headers = headers)

data = response.json()

In [73]:
summoner_ids = []
for entry in data["entries"]:
    summoner_ids.append(entry["summonerId"])

In [51]:
base_url = 'https://na1.api.riotgames.com/tft/summoner/v1/summoners/'

summoners = []

for ID in summoner_ids:
    url = base_url + ID + '?api_key=' + API_KEY
    response = requests.get(url=url, headers=headers)
    summoners.append(response.json())
    time.sleep(1) 

In [54]:
puuids = [summoner["puuid"] for summoner in summoners]

In [59]:
base_url = 'https://americas.api.riotgames.com/tft/match/v1/matches/by-puuid/'

match_ids = []

for puuid in puuids: 
    url = base_url + puuid + '/ids?start=0&count=10&api_key=' + API_KEY
    response = requests.get(url = url, headers=headers)
    match_ids.append(response.json())
    time.sleep(1)

In [63]:
# 1. Flatten the list of lists into a single list
flattened = [item for sublist in match_ids for item in sublist]

In [65]:
# 2a. Remove duplicates without preserving order:
unique_ids_set = list(set(flattened))
print(len(unique_ids_set))

1489


In [66]:
unique_ids_set[0]

'NA1_5214474372'

In [67]:
base_url = 'https://americas.api.riotgames.com/tft/match/v1/matches/'

match_data = []

for match_id in unique_ids_set: 
    url = base_url + match_id + '?api_key=' + API_KEY
    response = requests.get(url = url, headers=headers)
    match_data.append(response.json())
    time.sleep(1)

In [70]:
len(match_data)

1489

In [71]:
with open("match_data.json", "w") as file:
    json.dump(match_data, file, indent=4)

In [2]:
import json
import pandas as pd

with open('match_data.json', 'r') as f:
    data = json.load(f)

data_set13 = []
data_set4 = []


for entry in data:
    if 'info' in entry and 'tft_set_number' in entry['info']:
        if entry['info']['tft_set_number'] == 13:
            data_set13.append(entry)
        elif entry['info']['tft_set_number'] == 4:
            data_set4.append(entry)
            
with open('match_data_set13.json', 'w') as f:
    json.dump(data_set13, f, indent=4)

with open('match_data_set4.json', 'w') as f:
    json.dump(data_set4, f, indent=4)

In [6]:
import os

# --- Step 1: Load the match_data_set13.json file ---
with open('match_data_set13.json', 'r') as f:
    data_set13 = json.load(f)

# --- Step 2: Group entries by tft_set_game_type ---
grouped_entries = {}

for entry in data_set13:
    # Ensure the entry has an 'info' section with 'tft_set_game_type'
    if 'info' in entry and 'tft_set_game_type' in entry['info']:
        game_type = entry['info']['tft_set_game_type']
        # If this game type is not already a key, add it
        if game_type not in grouped_entries:
            grouped_entries[game_type] = []
        grouped_entries[game_type].append(entry)
    else:
        print("Warning: Entry missing 'tft_set_game_type' in 'info'. Skipping entry:", entry)

# --- Step 3: Save each group to a separate JSON file ---
for game_type, entries in grouped_entries.items():
    # Create a safe file name by replacing spaces or special characters if needed.
    # Here we replace spaces with underscores and remove slashes.
    safe_game_type = game_type.replace(" ", "_").replace("/", "_")
    filename = f"match_data_set13_{safe_game_type}.json"
    with open(filename, 'w') as outfile:
        json.dump(entries, outfile, indent=4)
    print(f"Saved {len(entries)} entries to {filename}")

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

